# Анализ отзывов отелей
## Прогнозирование удовлетворенности гостей и выявление ключевых факторов

---



## 1. Введение и Постановка задачи
### Бизнес-контекст
Для отелей и платформ бронирования понимание факторов, влияющих на удовлетворенность гостей, критически важно. Этот проект анализирует отзывы, чтобы выявить ключевые драйверы удовлетворенности и построить модель прогнозирования оценок.

**Основные вопросы:**
- Какие аспекты (персонал, чистота, удобства) больше всего влияют на общую оценку?
- Как демографические характеристики гостей влияют на их оценки?
- Можно ли предсказать оценку отеля на основе его характеристик и профиля гостя?

**Методы:** EDA, статистический анализ, машинное обучение (регрессия), интерпретация моделей.

---


## 2. Настройка окружения

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Машинное обучение
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.inspection import permutation_importance
from sklearn.impute import SimpleImputer

# Статистика
from scipy import stats
from scipy.stats import ttest_ind, f_oneway

import warnings
warnings.filterwarnings('ignore')

# Настройка стилей для визуализаций
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

---

## 3. Загрузка данных из SQLite базы

In [2]:
def load_data_from_db(db_path):
    """
    Загружает данные из SQLite базы данных
    """
    try:
        # Подключаемся к базе данных
        conn = sqlite3.connect(db_path)
        
        # Загружаем таблицы в DataFrame
        hotels_df = pd.read_sql_query("SELECT * FROM hotels", conn)
        reviews_df = pd.read_sql_query("SELECT * FROM reviews", conn)
        users_df = pd.read_sql_query("SELECT * FROM users", conn)
        
        conn.close()
        
        print(f"    Данные успешно загружены:")
        print(f"   - Hotels: {hotels_df.shape}")
        print(f"   - Reviews: {reviews_df.shape}")
        print(f"   - Users: {users_df.shape}")
        
        return hotels_df, reviews_df, users_df
        
    except Exception as e:
        print(f"Ошибка при загрузке данных: {e}")
        return None, None, None

# Загружаем данные
db_path = "booking_db.sqlite"  # укажите путь к данным
hotels, reviews, users = load_data_from_db(db_path)

    Данные успешно загружены:
   - Hotels: (25, 13)
   - Reviews: (50000, 12)
   - Users: (2000, 6)


---

## 4. Первичный осмотр данных

In [3]:
def explore_dataframe(df, name="DataFrame"):
    """
    Функция для первичного осмотра данных одного DataFrame.
    
    Параметры:
    - df: pandas.DataFrame — DataFrame для анализа.
    - name: str — имя таблицы или описание для удобства чтения.
    """
    print(f" ПЕРВИЧНЫЙ ОСМОТР ДАННЫХ: {name.upper()}")
    print("=" * 50)
    
    # Вывод первых строк
    print("\nПервые 3 строки:")
    print(df.head(3))
    
    # Информация о таблице
    print("\nИнформация о таблице:")
    df.info()
    
    # Статистика числовых столбцов
    print("\nСтатистика числовых столбцов:")
    print(df.describe())

In [4]:
# Исследуем hotels
explore_dataframe(hotels, name="Hotels")

 ПЕРВИЧНЫЙ ОСМОТР ДАННЫХ: HOTELS

Первые 3 строки:
   hotel_id         hotel_name      city         country  star_rating  \
0         1    The Azure Tower  New York   United States            5   
1         2  The Royal Compass    London  United Kingdom            5   
2         3    L'Étoile Palace     Paris          France            5   

       lat      lon  cleanliness_base  comfort_base  facilities_base  \
0  40.7580 -73.9855               9.1           8.8              8.9   
1  51.5072  -0.1276               9.0           9.2              8.8   
2  48.8566   2.3522               8.8           9.4              8.7   

   location_base  staff_base  value_for_money_base  
0            9.5         8.6                   8.0  
1            9.4         9.0                   7.9  
2            9.6         9.3                   8.1  

Информация о таблице:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 13 columns):
 #   Column                No

In [5]:
# Исследуем reviews
explore_dataframe(reviews, name = "Rewiews")

 ПЕРВИЧНЫЙ ОСМОТР ДАННЫХ: REWIEWS

Первые 3 строки:
   review_id  user_id  hotel_id review_date  score_overall  score_cleanliness  \
0          1     1600         1  2022-10-07            8.7                8.6   
1          2      432         4  2020-03-24            9.1               10.0   
2          3      186        18  2023-12-18            8.8                9.7   

   score_comfort  score_facilities  score_location  score_staff  \
0            8.7               8.5             9.0          8.8   
1            9.1               9.0             8.6          9.4   
2            8.8               8.3             8.7          8.1   

   score_value_for_money                                        review_text  
0                    8.7  Practice reduce young our because machine. Rec...  
1                    8.6  Test cover traditional black. Process tell Mr ...  
2                    8.6  Friend million student social study yeah. Grow...  

Информация о таблице:
<class 'pandas.core

In [6]:

# Исследуем users
explore_dataframe(users, name="Users")

 ПЕРВИЧНЫЙ ОСМОТР ДАННЫХ: USERS

Первые 3 строки:
   user_id user_gender         country age_group traveller_type   join_date
0        1      Female  United Kingdom     35-44           Solo  2024-09-29
1        2        Male  United Kingdom     25-34           Solo  2023-11-29
2        3      Female          Mexico     25-34         Family  2022-04-03

Информация о таблице:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   user_id         2000 non-null   int64 
 1   user_gender     2000 non-null   object
 2   country         2000 non-null   object
 3   age_group       2000 non-null   object
 4   traveller_type  2000 non-null   object
 5   join_date       2000 non-null   object
dtypes: int64(1), object(5)
memory usage: 93.9+ KB

Статистика числовых столбцов:
           user_id
count  2000.000000
mean   1000.500000
std     577.494589
min    

---

## 5. Проверка на пропуски и дубликаты

In [7]:
def check_data_quality(df, name):
    """
    Проверка качества данных для одного DataFrame.

    Параметры:
    - df: pandas.DataFrame — DataFrame для анализа.
    - name: str — имя таблицы или описание для удобства чтения.
    """

    print(f"ПРОВЕРКА КАЧЕСТВА ДАННЫХ: {name.upper()}")
    print("=" * 50)

    # Размер данных 
    print(f"\n{name}")
    print(f"    Размер: {df.shape}")

    # Проверка на пропуски 
    print(f"    Пропуски: {df.isnull().sum().sum()}")
    if df.isnull().sum().sum() > 0:
        print(f"    Столбцы с пропусками:")
        for col in df.columns:
            null_count = df[col].isnull().sum()
            if null_count > 0:
                print(f"    -{col}: {null_count} ({null_count / len(df) * 100:.2f}%)")

    # Проверка на дубликаты
    print(f"   Дубликаты: {df.duplicated().sum()}")
    
    # Проверка уникальных значений ключевых столбцов
    key_columns = ['hotel_id', 'review_id', 'user_id']
    for col in key_columns:
        if col in df.columns:
            print(f"   Уникальных значений в '{col}': {df[col].nunique()}")

In [8]:
check_data_quality(hotels, name="Hotels")

ПРОВЕРКА КАЧЕСТВА ДАННЫХ: HOTELS

Hotels
    Размер: (25, 13)
    Пропуски: 0
   Дубликаты: 0
   Уникальных значений в 'hotel_id': 25


In [9]:
check_data_quality(reviews, name="Reviews")

ПРОВЕРКА КАЧЕСТВА ДАННЫХ: REVIEWS

Reviews
    Размер: (50000, 12)
    Пропуски: 0
   Дубликаты: 0
   Уникальных значений в 'hotel_id': 25
   Уникальных значений в 'review_id': 50000
   Уникальных значений в 'user_id': 2000


In [10]:
check_data_quality(users, name="Users")

ПРОВЕРКА КАЧЕСТВА ДАННЫХ: USERS

Users
    Размер: (2000, 6)
    Пропуски: 0
   Дубликаты: 0
   Уникальных значений в 'user_id': 2000
